In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG

from IPython.display import Image, display
from sklearn.model_selection import train_test_split


In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label


In [8]:
accuracy = []

N_material = 61
N_material_train = 40

for fold in range(10):
    
    idx_train = np.array([])
    for i in range(N_material_train):
        train_class = np.random.randint(0, N_material)
        idx_train= np.concatenate((idx_train, np.where(label_material == train_class)[0]))
    
    # need to get better code later
    idx_test = np.setdiff1d(np.arange(N), idx_train)
    
    idx_train = idx_train.astype(int)
    idx_test = idx_test.astype(int)
    
    X_train = raw_feature[idx_train,:]
    Y_VSP_train = label_visiprog[idx_train]
    Y_mat_train = label_material[idx_train]

    X_test = raw_feature[idx_test,:]
    Y_VSP_test = label_visiprog[idx_test]
    Y_mat_test = label_material[idx_test]

    model = LFDA_VISIPROG()
    model.fit(X_train, Y_VSP_train)

    X_vsp_test = model.transform(X_test)

    res = leave_one_sample_out(X_vsp_test, Y_mat_test)
    accuracy.append(res['accuracy'])

(3261,)
(2719,)


In [9]:
display(accuracy)

[0.99448326590658331]

In [10]:
np.array(accuracy).mean()

0.99448326590658331